In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!pip install pyspark==3.5.0 -q
!pip install gcsfs -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!wget https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.7/gcs-connector-hadoop3-2.2.7-shaded.jar -P /usr/local/lib/

--2025-01-23 19:56:06--  https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.7/gcs-connector-hadoop3-2.2.7-shaded.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33831577 (32M) [application/java-archive]
Saving to: ‘/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar.1’

gcs-connector-hadoo 100%[===================>]  32.26M   197MB/s    in 0.2s    

2025-01-23 19:56:06 (197 MB/s) - ‘/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar.1’ saved [33831577/33831577]



In [1]:
!gsutil ls gs://edit-de-project-streaming-data/carris-vehicles/

ServiceException: 401 Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist).


In [ ]:
!gsutil cat gs://edit-de-project-streaming-data/carris-vehicles/vehicles_1735510622.json

In [3]:
!rm -rf /content/output/*

In [4]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=paepnIJuJ2Kk34XtHHmL7NnAOC8mw7&prompt=consent&token_usage=remote&access_type=offline&code_challenge=5x-rrYdXQ052gceM2zuU4_Omris05UZYePemw0Fn3CY&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AanRRrttfCgZd6-kcAAChB1xhAPxQPfyOKBmL9N_zORog0Ehdb1mfSons2EbLR612jJDJA

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [6]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, LongType
from pyspark.sql.functions import col, avg, count, sum, lit
import time

# Create session Spark with conector GCS
spark = SparkSession.builder \
    .appName("GCSReadVehicles") \
    .config("spark.jars", "/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .getOrCreate()

# Default location of file generated by gcloud
credential_path = "/content/.config/application_default_credentials.json"

# Ambient variable configuration of credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_path

# Config PySpark to access the GCS
spark._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
spark._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
spark._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile", credential_path)


# Bucket Path
bucket_path = "gs://edit-de-project-streaming-data/carris-vehicles/"


# Define schema (Used printSchema())
schema = StructType([
    StructField("bearing", LongType(), True),
    StructField("block_id", StringType(), True),
    StructField("current_status", StringType(), True),
    StructField("id", StringType(), True),
    StructField("lat", DoubleType(), True),
    StructField("line_id", StringType(), True),
    StructField("lon", DoubleType(), True),
    StructField("pattern_id", StringType(), True),
    StructField("route_id", StringType(), True),
    StructField("schedule_relationship", StringType(), True),
    StructField("shift_id", StringType(), True),
    StructField("speed", DoubleType(), True),
    StructField("stop_id", StringType(), True),
    StructField("timestamp", LongType(), True),
    StructField("trip_id", StringType(), True),
])

# Reading Json data from GCS
df = spark.readStream \
    .format("json") \
    .schema(schema) \
    .option("maxFilesPerTrigger", 5) \
    .load("gs://edit-de-project-streaming-data/carris-vehicles/")

# Path of folder to keep output from bucket
output_path = "/content/output"
#output_path = "gs://edit-data-eng-project-group2/vehicles_data/"

# Writing in query and saving in output path
query = df.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", output_path) \
    .option("checkpointLocation", "/content/output/checkpoints/") \
    .trigger(processingTime="10 seconds") \
    .start()
#.option("checkpointLocation", "gs://edit-data-eng-project-group2/checkpoints/") \
# 30 seconds of execution just for testing
#time.sleep(30)
#query.stop()


In [5]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, LongType
from pyspark.sql.functions import col, count, sum, lit, window, first, sqrt, pow, last
from pyspark.sql.window import Window
import time

# Cria a sessão Spark com o conector GCS
spark = SparkSession.builder \
    .appName("GCSReadVehicles") \
    .config("spark.jars", "/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .getOrCreate()

# Caminho do arquivo de credenciais gerado pelo gcloud
credential_path = "/content/.config/application_default_credentials.json"

# Configuração do ambiente para as credenciais
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_path

# Configura o PySpark para acessar o GCS
spark._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
spark._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
spark._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile", credential_path)

# Caminho do bucket no GCS
bucket_path = "gs://edit-de-project-streaming-data/carris-vehicles/"

# Define o esquema (usado para printSchema())
schema = StructType([
    StructField("bearing", LongType(), True),
    StructField("block_id", StringType(), True),
    StructField("current_status", StringType(), True),
    StructField("id", StringType(), True),
    StructField("lat", DoubleType(), True),
    StructField("line_id", StringType(), True),
    StructField("lon", DoubleType(), True),
    StructField("pattern_id", StringType(), True),
    StructField("route_id", StringType(), True),
    StructField("schedule_relationship", StringType(), True),
    StructField("shift_id", StringType(), True),
    StructField("speed", DoubleType(), True),
    StructField("stop_id", StringType(), True),
    StructField("timestamp", LongType(), True),
    StructField("trip_id", StringType(), True),
])

# 1ª Query: Lê os dados do GCS e escreve no diretório output/bronze
df = spark.readStream \
    .format("json") \
    .schema(schema) \
    .option("maxFilesPerTrigger", 5) \
    .load(bucket_path)

# Caminho da pasta onde os dados brutos serão gravados
bronze_output_path = "output/bronze"

# Garantir que a coluna timestamp esteja no formato correto
df = df.withColumn("timestamp", col("timestamp").cast("timestamp"))

# Definir a janela de 2 minutos para agregações
agg_distance = df.withWatermark("timestamp", "2 minutes") \
    .groupBy("id", window(col("timestamp"), "2 minutes")) \
    .agg(
        first("lat").alias("start_lat"),
        first("lon").alias("start_lon"),
        last("lat").alias("end_lat"),
        last("lon").alias("end_lon"),
        count("*").alias("data_points")
    )

# Calcular a distância utilizando a fórmula de distância euclidiana entre o primeiro e o último ponto na janela
agg_distance = agg_distance.withColumn(
    "distance",
    sqrt(
        pow(col("end_lat") - col("start_lat"), 2) + pow(col("end_lon") - col("start_lon"), 2)
    ) * lit(111)  # Aproximação simplificada (em km)
)

# Calcular a velocidade média (km/h) com base na distância e tempo
agg_speed = agg_distance.withColumn(
    "average_speed_kmh",
    (col("distance") / lit(2 / 60))  # Tempo de 2 minutos convertido para horas
)

# Agregar atributos do veículo na janela de 2 minutos
windowed_df = df.withWatermark("timestamp", "2 minutes") \
    .groupBy("id", window(col("timestamp"), "2 minutes")) \
    .agg(
        first("line_id").alias("line"),
        first("route_id").alias("route"),
        first("bearing").alias("direction"),
        first("stop_id").alias("next_stop"),
        count("*").alias("data_points")
    )

windowed_df = windowed_df.drop("data_points")

# Juntar os dados de velocidade e distância com os atributos do veículo
final_result = agg_speed.join(windowed_df, ["id", "window"], "inner")

final_result = final_result.drop("data_points")
final_result = final_result.drop("start_lat")
final_result = final_result.drop("start_lon")
final_result = final_result.drop("end_lat")
final_result = final_result.drop("end_lon")

# Grava no diretório output/bronze
query_bronze = final_result.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", bronze_output_path) \
    .option("checkpointLocation", f"{bronze_output_path}/_checkpoint") \
    .start()

# Defina o esquema para os dados Parquet
schema = StructType([
    StructField("id", StringType(), True),
    StructField("start_lat", DoubleType(), True),
    StructField("start_lon", DoubleType(), True),
    StructField("end_lat", DoubleType(), True),
    StructField("end_lon", DoubleType(), True),
    StructField("data_points", LongType(), True),
    StructField("distance", DoubleType(), True),
    StructField("average_speed_kmh", DoubleType(), True),
    StructField("line", StringType(), True),
    StructField("route", StringType(), True),
    StructField("direction", StringType(), True),
    StructField("next_stop", StringType(), True),
    StructField("window", StringType(), True)
])

# Ler o arquivo Parquet em modo streaming
#parquet_df_stream = spark.readStream \
#    .schema(schema) \
#    .parquet("output/silver")

# Exibir os primeiros registros (em modo streaming, apenas enquanto processa)
#parquet_df_stream.writeStream \
#    .outputMode("append") \
#    .format("console") \
#    .start()



time.sleep(240)
query_bronze.stop()
# Aguarda até o término do streaming
#query_bronze.awaitTermination()
#query_silver.awaitTermination()
# Aguarda até o término do streaming
#parquet_df_stream.awaitTermination()


In [8]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Criar a sessão Spark
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# Caminho para o arquivo Parquet específico
file_path = "/content/output/silver/part-00000-2b2120f2-6cb2-4270-9ace-cd8e76523b0f-c000.snappy.parquet"

# Ler o arquivo Parquet específico
parquet_df = spark.read.parquet(file_path)

# Exibir os primeiros registros
parquet_df.show()



AnalysisException: [COLUMN_ALREADY_EXISTS] The column `data_points` already exists. Consider to choose another name or rename the existing column.

In [18]:
import pyarrow.parquet as pq
import pandas as pd

# Caminho para o arquivo Parquet
file_path = "/content/output/silver/part-00000-a181ef3e-f1ae-4aa8-bcbf-d8e667a800f6-c000.snappy.parquet"

# Ler o arquivo Parquet
parquet_file = pq.ParquetFile(file_path)

# Exibir o esquema completo para melhor compreensão
print(parquet_file.schema)

# Ler o conteúdo do arquivo
table = parquet_file.read()

# Converter para DataFrame do Pandas e exibir os dados
df = table.to_pandas()

# Exibir as primeiras linhas
print(df.head())



required group field_id=-1 spark_schema {
  optional binary field_id=-1 id (String);
  required group field_id=-1 window {
    optional int96 field_id=-1 start;
    optional int96 field_id=-1 end;
  }
  optional double field_id=-1 start_lat;
  optional double field_id=-1 start_lon;
  optional double field_id=-1 end_lat;
  optional double field_id=-1 end_lon;
  required int64 field_id=-1 data_points;
  optional double field_id=-1 distance;
  optional double field_id=-1 average_speed_kmh;
  optional binary field_id=-1 line (String);
  optional binary field_id=-1 route (String);
  optional int64 field_id=-1 direction;
  optional binary field_id=-1 next_stop (String);
  required int64 field_id=-1 data_points;
}

Empty DataFrame
Columns: [id, window, start_lat, start_lon, end_lat, end_lon, data_points, distance, average_speed_kmh, line, route, direction, next_stop, data_points]
Index: []


In [18]:
from pyspark.sql.window import Window
from pyspark.sql.functions import from_unixtime,timestamp_seconds, udf, col, lag, radians, sin, cos, sqrt, atan2

def transform_data():
    # Adicionar coluna transformada
    batch_df = batch_df.withColumn("timestamp", timestamp_seconds(col("timestamp")))

    # Criar janela para calcular lag
    vehicle_window = Window.partitionBy("id").orderBy("timestamp")


    # Adicionar colunas de latitude e longitude anteriores
    batch_df = batch_df.withColumn("prev_lat", lag("lat").over(vehicle_window))
    batch_df = batch_df.withColumn("prev_lon", lag("lon").over(vehicle_window))

    # Escrever o resultado no diretório em JSON
    batch_df.write.mode("append").format("parquet").save("/content/silver/vehicles")



In [ ]:

# Perform transformations to calculate metrics
agg_df = df.groupBy("route_id").agg(
    avg("speed").alias("average_speed"),
    count("trip_id").alias("number_of_trips"),
    sum(lit(2) * col("speed")).alias("total_distance_km"),  # Approximation assuming speed is in km/h
    sum(col("timestamp")).alias("total_travel_time")
)

# Display aggregated data to console
query = agg_df.writeStream \
    .outputMode("complete") \
    .format("console") \
    .trigger(processingTime="10 seconds") \
    .start()

# Path of folder to keep aggregated output
#aggregated_output_path = "gs://edit-data-eng-project-group2/aggregated_metrics/"

In [11]:
df = spark.readStream \
    .format("json") \
    .schema(schema) \
    .option("maxFilesPerTrigger", 5) \
    .load("/content/output/")

In [19]:
query = (df.writeStream
         .outputMode('append')
         .format('parquet')
         .option('checkpointLocation', '/content/silver/vehicles_checkpoint')
         .trigger(processingTime='5 seconds')
         .foreachBatch(transform_data)
         .start()
)

In [20]:
query.isActive

True

In [7]:
output_df = spark.read.format("parquet").load("/content/output/bronze")
output_df.show()

+--------+--------------------+--------------------+------------------+----+------+---------+---------+
|      id|              window|            distance| average_speed_kmh|line| route|direction|next_stop|
+--------+--------------------+--------------------+------------------+----+------+---------+---------+
| 41|1125|{2025-01-21 17:56...| 0.23853682262448198|  7.15610467873446|1704|1704_0|      217|   030783|
| 41|1349|{2025-01-21 17:56...| 0.22316170332161805| 6.694851099648542|1527|1527_0|      303|   120279|
| 41|1401|{2025-01-21 17:56...| 0.28088922979385883| 8.426676893815765|1254|1254_0|      146|   171461|
| 41|1800|{2025-01-21 17:56...| 0.05519986952938248|1.6559960858814744|1229|1229_0|        0|   170971|
|  41|335|{2025-01-21 17:56...| 0.29367113808482587| 8.810134142544776|1719|1719_0|      230|   030626|
| 42|2018|{2025-01-21 17:56...|                 0.0|               0.0|2926|2926_0|      249|   200002|
| 42|2300|{2025-01-21 17:56...|                 0.0|            

In [ ]:
# Path to file in colab
file_path = "/content/output/part-00000-fc32eaa6-deef-4f2c-b72b-2009f077bb01-c000.json"

# Loading Json into Dataframe Spark
df = spark.read.json(file_path)

# Showing schema of dataframe
df.printSchema()

# Showing data
df.show()

root
 |-- bearing: long (nullable = true)
 |-- block_id: string (nullable = true)
 |-- current_status: string (nullable = true)
 |-- id: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- line_id: string (nullable = true)
 |-- lon: double (nullable = true)
 |-- pattern_id: string (nullable = true)
 |-- route_id: string (nullable = true)
 |-- schedule_relationship: string (nullable = true)
 |-- shift_id: string (nullable = true)
 |-- speed: double (nullable = true)
 |-- stop_id: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- trip_id: string (nullable = true)

+-------+--------------------+--------------+--------+------------------+-------+------------------+----------+--------+---------------------+------------+------------------+-------+----------+--------------------+
|bearing|            block_id|current_status|      id|               lat|line_id|               lon|pattern_id|route_id|schedule_relationship|    shift_id|             speed|stop_id| t